# Easy Agent Tutorial
This notebook file provide three examples of using LLM based agents with different tool sets.

prequisites:
- Python 3.10+
- Install required packages:
  ```bash
  pip install -r "mcp[cli]" smolagent
  ```

## SmolAgent::CodeAgent

In [ ]:
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from smolagents import CodeAgent, OpenAIServerModel, tool

load_dotenv()

db = FAISS.load_local(
    "faiss_db",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)

with open("keywords.txt", "r", encoding="utf-8") as f:
    keywords = f.read().strip()
content = f"$SYS_PROMPT"


@tool
def get_keywords() -> str:
    """
    return the keywords for RAG search
    """
    return " ".join(keywords.split())


@tool
def query_db(kw: str) -> str:
    """
    accept keyword and return related information

    Args:
        kw (str): The keyword to query the database
    """
    docs = []
    for skw in kw.split():
        if not (skw := skw.strip()):
            continue
        print(f"Searching for keyword: {skw}")
        docs.extend(db.similarity_search(skw, k=3))
    rag_result = "\n".join([doc.page_content for doc in docs])
    return rag_result


agent = CodeAgent(
    model=OpenAIServerModel("gpt-5"),
    tools=[get_keywords, query_db],  # add your tools here (don't remove final_answer)
    verbosity_level=1,
)

# from Gradio_UI import GradioUI

# GradioUI(agent).launch()

agent.run(f"{content}\n\n$USER_PROMPT")